In [1]:
import numpy as np
import pandas as pd
import requests
import tqdm
import time
import random
import re

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

from time import sleep

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#pd.reset_option('display.max_rows')

In [4]:
url = "https://www.autocasion.com/coches-ocasion"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

session = requests.Session()

response = session.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")
print(response.status_code)

200


In [5]:
anuncios_pagina = soup.find_all("article", class_="anuncio")

total_coches = int(soup.find('div', class_="container listado").find('h1').text.strip().split('\n')[0].replace('.',''))

total_paginas = int(total_coches/len(anuncios_pagina)) + (1 if (total_coches/len(anuncios_pagina))%1 else 0)

total_paginas

6630

In [6]:
def links_coches():
    url_base = "https://www.autocasion.com"
    url_pagina = f"{url_base}/coches-ocasion?direction=desc&sort=updated_at&page="
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    session = requests.Session()
    
    lista_paginas = []
    anuncios_coches = []

    for pagina in range(1, total_paginas +1):
        lista_paginas.append(f'{url_pagina}{pagina}')
        
    lista_prueba = lista_paginas[:5]
        
    for pagina_url in lista_prueba:
        response = session.get(pagina_url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        anuncios = soup.find_all("article", class_="anuncio")
        
        for anuncio in anuncios:
            coche_url = anuncio.find("a")['href']
            anuncios_coches.append(f'{url_base}{coche_url}')   
    
        
    return lista_paginas, anuncios_coches

In [7]:
lista_paginas,anuncios_coches = links_coches()
anuncios_coches

['https://www.autocasion.com/coches-segunda-mano/porsche-911-ocasion/911-carrera-cabriolet-pdk-2-ref14328716',
 'https://www.autocasion.com/coches-segunda-mano/porsche-panamera-ocasion/panamera-turbo-e-hybrid-aut-ref14577154',
 'https://www.autocasion.com/coches-segunda-mano/dacia-sandero-ocasion/sandero-1-5dci-laureate-75-2-ref9517321',
 'https://www.autocasion.com/coches-km0/km-0/volkswagen-t-roc-km0/t-roc-1-5-tsi-110kw-dsg-life-150-5p-16-ref13530038',
 'https://www.autocasion.com/coches-segunda-mano/citroen-c3-ocasion/c3-1-2-puretech-feel-82-ref14706890',
 'https://www.autocasion.com/coches-segunda-mano/skoda-kamiq-ocasion/kamiq-1-0-tsi-ambition-81kw-ref14137724',
 'https://www.autocasion.com/coches-segunda-mano/peugeot-208-ocasion/208-active-electrico-100kw-ref14352272',
 'https://www.autocasion.com/coches-segunda-mano/skoda-karoq-ocasion/2-0-tdi-85kw-ambition-115-5p-166f66e1d80916-ref14430775',
 'https://www.autocasion.com/coches-segunda-mano/citroen-c3-ocasion/c3-1-2-puretech-fee

In [9]:
headers = {
           "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
            }
session = requests.Session()

In [10]:
# Función para buscar y extraer un valor específico de cada cadena
def buscar_detalle(texto, campo):
    # Usamos expresiones regulares para capturar el campo deseado seguido de su valor
    pattern = rf"{campo}\s*:\s*([^\n]+)"
    match = re.search(pattern, texto)
    return match.group(1).strip() if match else None

In [14]:
coches_df = dict()

for coche in tqdm.tqdm(anuncios_coches):
    print(coche)
    url_coche = coche
    response = session.get(url_coche, headers=headers)
    
    #Verificamos que la respuesta es exitosa
    if response.status_code != 200:
        print(f"Error al acceder a {url_coche}, Código de estado: {response.status_code}")
        continue
    
    soup = BeautifulSoup(response.content, "html.parser")
    info_coche = {}
        
    info_coche['provincia'] = soup.find_all('div', class_= 'etiquetas')[1].find_all('span')[4].text
    info_coche['marca_modelo'] = soup.find('h2').text
    info_coche['fecha_matriculacion'] = soup.find('ul', class_= 'datos-basicos-ficha').find('span', class_= 'icon icon-calendario').find_previous().text.strip()
    info_coche['kilometraje'] = soup.find('ul', class_= 'datos-basicos-ficha').find('span', class_= 'icon icon-velocimetro').find_previous().text.replace('.','').replace('km','').strip()
    
    lista_detalles_vendedor = soup.find('div', class_= 'datos-concesionario').find('p').text.strip().split('\n')
    info_coche['nombre_vendedor'] = lista_detalles_vendedor[0]
    info_coche['tipo_vendedor'] = lista_detalles_vendedor[1].strip()
    try:
        #puede haber o no
        info_coche['distintivo_ambiental'] = soup.find_all('div', class_= 'etiquetas')[1].find('img')['alt']
        
    except:
        info_coche['distintivo_ambiental'] = 'no disponible'
    try:
        #puede haber o no
        info_coche['garantia'] = soup.find('ul', class_= 'datos-basicos-ficha').find('span', class_= 'icon icon-file').find_previous().text.replace('Garantía: ', '').strip()
        
    except:
        info_coche['garantia'] = 'sin garantía'
    
    
    detalles_precio = soup.find('ul', class_= 'tabla-precio').find_all('li')

    for detalle in detalles_precio:
        detalle = detalle.text.replace('\n','').strip()

        if 'Precio al contado' in detalle: 
            info_coche['precio_contado'] = detalle.replace('\n', '').split(':')[1].replace('.', '').replace('€','').strip()
        elif 'Precio de venta nuevo' in detalle: 
            info_coche['precio_nuevo'] = detalle.replace('\n', '').split('*')[1].strip().replace('.','').split()[0]
            
    if 'precio_nuevo' not in info_coche:
        info_coche['precio_nuevo'] = 'no disponible'
    
    div_ficha_tecnica = soup.find('div', class_='bloque ficha-tecnica') 
    div_no_eurotax = soup.find('div', class_= 'bloque no-eurotax')
    
    if div_ficha_tecnica:
        
        detalles_carroceria = div_ficha_tecnica.find('ul', class_= 'tab-spec-1 active').find_all('li')
        for detalle in detalles_carroceria:
            detalle = detalle.text.replace('Medidas','').strip()
            
            if 'Largo' in detalle: 
                info_coche['largo'] = detalle
            elif 'Ancho' in detalle: 
                info_coche['ancho'] = detalle
            elif 'Alto' in detalle: 
                info_coche['alto'] = detalle
            #el primer número es la capacidad del maletero normal, el segundo la capacidad del maletero con los asientos abatidos
            elif 'Capacidad del maletero' in detalle: 
                info_coche['capacidad_maletero'] = detalle
            elif 'Carrocería' in detalle: 
                info_coche['carroceria'] = detalle
            elif 'Número de plazas' in detalle: 
                info_coche['num_plazas'] = detalle
            elif 'Número de puertas' in detalle: 
                info_coche['num_puertas'] = detalle
            elif 'Batalla' in detalle: 
                info_coche['batalla'] = detalle
            #supongo que si hay dos pesos el segundo será la masa max. autorizada
            elif 'Peso' in detalle: 
                info_coche['peso'] = detalle
           
        detalles_consumos = div_ficha_tecnica.find('ul', class_= 'tab-spec-2').find_all('li')
        for detalle in detalles_consumos:
            detalle = ' '.join(detalle.text.replace('\n','').split())
            
            if 'Consumo medio' in detalle: 
                info_coche['consumo_medio'] = detalle
            elif 'Consumo en carretera' in detalle: 
                info_coche['consumo_carretera'] = detalle
            elif 'Consumo urbano' in detalle: 
                info_coche['consumo_urbano'] = detalle
            elif 'CO2' in detalle: 
                info_coche['CO2'] = detalle
            elif 'Depósito' in detalle: 
                info_coche['deposito'] = detalle

        detalles_motor = div_ficha_tecnica.find('ul', class_= 'tab-spec-3').find_all('li')
        for detalle in detalles_motor:
            detalle = detalle.text.replace('\n', ' ').strip()
            
            if 'Combustible' in detalle: 
                info_coche['combustible'] = detalle
            elif 'Nº de cilindros' in detalle: 
                info_coche['cilindros'] = detalle
            elif 'Cilindrada' in detalle: 
                info_coche['cilindrada'] = detalle
            elif 'Sobrealimentación' in detalle: 
                info_coche['sobrealimentacion'] = detalle

        detalles_transmision = div_ficha_tecnica.find('ul', class_= 'tab-spec-4').find_all('li')
        for detalle in detalles_transmision:
            detalle = detalle.text.replace('\n', ' ').strip()
            
            if 'Tipo de transmisión' in detalle: 
                info_coche['tipo_cambio'] = detalle
            elif 'Tracción' in detalle: 
                info_coche['traccion'] = detalle
            elif 'Número de marchas' in detalle: 
                info_coche['num_marchas'] = detalle

        detalles_prestaciones = div_ficha_tecnica.find('ul', class_= 'tab-spec-5').find_all('li')
        for detalle in detalles_prestaciones:
            detalle = detalle.text
            
            if 'Potencia (KW)' in detalle: 
                info_coche['potencia_kw'] = detalle
            elif 'Potencia (CV)' in detalle: 
                info_coche['potencia_cv'] = detalle
            elif 'Par' in detalle: 
                info_coche['par'] = detalle
            elif 'Velocidad' in detalle: 
                info_coche['velocidad_max'] = detalle
            elif 'Aceleración' in detalle: 
                info_coche['aceleracion'] = detalle
                
    elif div_no_eurotax: 
        
        ficha_no_eurotax = div_no_eurotax.find_all('ul')
        lista_ficha_no_eurotax = []
        for elemento in range(0,len(ficha_no_eurotax)):
            detalle = div_no_eurotax.find_all('ul')[elemento].text
    
            if "Carrocería" in detalle:
                info_coche['carroceria'] = buscar_detalle(detalle, "Carrocería")
                info_coche['num_puertas'] = buscar_detalle(detalle, "Número de puertas")
                info_coche['num_plazas'] = buscar_detalle(detalle, "Número de plazas")
                info_coche['peso'] = buscar_detalle(detalle, "Peso")

            elif "Consumo y emisiones" in detalle: 
                info_coche['consumo_medio']= buscar_detalle(detalle, "Medio \(l/100 km\)")
                info_coche['consumo_carretera']= buscar_detalle(detalle, "Carretera \(l/100 km\)")
                info_coche['consumo_urbano']= buscar_detalle(detalle, "Urbano \(l/100 km\)")

            elif "Motor" in detalle:
                info_coche['combustible'] = buscar_detalle(detalle, "Combustible")
                info_coche['potencia_cv'] = buscar_detalle(detalle, "Potencia")
                info_coche['cilindrada'] = buscar_detalle(detalle, "Cilindrada")
            elif "Transmisión" in detalle:
                info_coche['tipo_cambio'] = buscar_detalle(detalle, "Tipo de cambio")
                info_coche['num_marchas'] = buscar_detalle(detalle, "Marchas")
                
    coches_df[(len(coches_df))] = info_coche
    
    time.sleep(random.uniform(1, 3))


  0%|          | 0/80 [00:00<?, ?it/s]

https://www.autocasion.com/coches-segunda-mano/porsche-911-ocasion/911-carrera-cabriolet-pdk-2-ref14328716


  1%|▏         | 1/80 [00:03<04:11,  3.19s/it]

https://www.autocasion.com/coches-segunda-mano/porsche-panamera-ocasion/panamera-turbo-e-hybrid-aut-ref14577154


  2%|▎         | 2/80 [00:04<02:40,  2.05s/it]

...

 98%|█████████▊| 78/80 [03:21<00:04,  2.38s/it]

https://www.autocasion.com/coches-segunda-mano/hyundai-ioniq-ocasion/ioniq-phev-1-6-gdi-klass-141cv-166f5593590a50-ref14421887


 99%|█████████▉| 79/80 [03:24<00:02,  2.68s/it]

https://www.autocasion.com/coches-segunda-mano/peugeot-2008-ocasion/2008-1-2-puretech-s-s-signature-82-ref14672146


100%|██████████| 80/80 [03:26<00:00,  2.58s/it]


In [15]:
df = pd.DataFrame.from_dict(coches_df).T

df.iloc[7]

provincia                                     Pontevedra
marca_modelo                Detalles SKODA Karoq (2022) 
fecha_matriculacion                 Matriculado: 05/2022
kilometraje                                        24127
nombre_vendedor         SUZUKI SKODA MOURENTE Pontevedra
tipo_vendedor                                Profesional
distintivo_ambiental                       no disponible
garantia                                              Sí
precio_contado                                     24990
precio_nuevo                               no disponible
largo                                                NaN
ancho                                                NaN
alto                                                 NaN
capacidad_maletero                                   NaN
carroceria                             4x4, SUV o pickup
num_plazas                                          None
batalla                                              NaN
peso                           

In [16]:
df

,provincia,marca_modelo,fecha_matriculacion,kilometraje,nombre_vendedor,tipo_vendedor,distintivo_ambiental,garantia,precio_contado,precio_nuevo,largo,ancho,alto,capacidad_maletero,carroceria,num_plazas,batalla,peso,num_puertas,consumo_medio,consumo_carretera,consumo_urbano,CO2,deposito,combustible,cilindros,cilindrada,sobrealimentacion,tipo_cambio,traccion,num_marchas,potencia_kw,potencia_cv,par,velocidad_max,aceleracion
0,Vizcaya,Detalles PORSCHE 911 (2020),Matriculado: 05/2020,21500,CENTRO PORSCHE BILBAO,Profesional,C,12 meses,135100 135700 0%,158158,4519 mm Largo,1852 mm Ancho,1297 mm Alto,Capacidad del maletero (l)132/0,CarroceríaConvertible,Número de plazas4,Batalla (mm)\n2.450,Peso/Masa max. autorizado (kg)2.015,Número de puertas2,"Consumo medio 9,60 litros/100 km","Consumo en carretera 7,60 litros/100 km","Consumo urbano 12,90 litros/100 km",CO2 (g/km) 218 g/km,"Depósito de combustible (l) 64,00 litros",Combustible Gasolina,Nº de cilindros 6 cilindros,Cilindrada (cm3) 2.981,Sobrealimentación Doble turbo,Tipo de transmisión Automática secuencial,TracciónTracción trasera,Número de marchas8,Potencia (KW)283,Potencia (CV)385,Par (Nm)450,Velocidad máxima (km/h)291,"Aceleración 0-100 km/h (s)4,40"
1,Vizcaya,Detalles PORSCHE Panamera (2024),Matriculado: 06/2024,10000,CENTRO PORSCHE BILBAO,Profesional,0 EMISIONES,24 meses,205700,204516,5054 mm Largo,1937 mm Ancho,1421 mm Alto,Capacidad del maletero (l)421/1.255,CarroceríaBerlina,Número de plazas4,Batalla (mm)\n2.950,Peso/Masa max. autorizado (kg)2.830,Número de puertas5,"Consumo medio 2,10 litros/100 km",Consumo en carreteralitros/100 km,Consumo urbanolitros/100 km,CO2 (g/km) 47 g/km,"Depósito de combustible (l) 80,00 litros",Combustible Gasolina y corriente eléctrica,Nº de cilindros 8 cilindros,Cilindrada (cm3) 3.996,Sobrealimentación Turbo de geometría variable,Tipo de transmisión Automática secuencial,TracciónTracción total permanente,Número de marchas8,Potencia (KW)500,Potencia (CV)680,Par (Nm)930,Velocidad máxima (km/h)315,"Aceleración 0-100 km/h (s)3,20"
2,Sevilla,Detalles DACIA Sandero (2017),Matriculado: 12/2017,124000,DURMOVIL,Profesional,C,12 meses,9499,9696,4069 mm Largo,1733 mm Ancho,1519 mm Alto,Capacidad del maletero (l)320/0,CarroceríaBerlina,Número de plazas5,Batalla (mm)\n2.589,Peso/Masa max. autorizado (kg)1.652,Número de puertas5,"Consumo medio 3,50 litros/100 km","Consumo en carretera 3,40 litros/100 km","Consumo urbano 3,70 litros/100 km",CO2 (g/km) 90 g/km,"Depósito de combustible (l) 50,00 litros",Combustible Diesel,Nº de cilindros 4 cilindros,Cilindrada (cm3) 1.461,Sobrealimentación Turbo,Tipo de transmisión Manual,TracciónTracción delantera,Número de marchas5,Potencia (KW)55,Potencia (CV)75,Par (Nm)200,Velocidad máxima (km/h)164,"Aceleración 0-100 km/h (s)14,60"
3,Barcelona,Detalles VOLKSWAGEN T-Roc (2024),Matriculado: 01/2024,10,Volkswagen Vilamòbil,Profesional,C,24 meses,33900,no disponible,NaN,NaN,NaN,NaN,"4x4, SUV o pickup",None,NaN,None,5 Puertas,NaN,NaN,NaN,NaN,NaN,Gasolina,NaN,None,NaN,Automático,NaN,None,NaN,150,NaN,NaN,NaN
4,Málaga,Detalles CITROEN C3 (2020),Matriculado: 10/2020,60000,MARBESOL,Profesional,C,12 meses,11190,15700,3996 mm Largo,1749 mm Ancho,1474 mm Alto,Capacidad del maletero (l)300/0,CarroceríaBerlina,Número de plazas5,Batalla (mm)\n2.539,Peso/Masa max. autorizado (kg)1.550,Número de puertas5,"Consumo medio 4,70 litros/100 km","Consumo en carretera 4,10 litros/100 km","Consumo urbano 5,70 litros/100 km",CO2 (g/km) 96 g/km,"Depósito de combustible (l) 45,00 litros",Combustible Gasolina,Nº de cilindros 3 cilindros,Cilindrada (cm3) 1.199,Sobrealimentación Turbo,Tipo de transmisión Manual,TracciónTracción delantera,Número de marchas5,Potencia (KW)60,Potencia (CV)82,Par (Nm)118,Velocidad máxima (km/h)168,"Aceleración 0-100 km/h (s)13,00"
5,Pontevedra,Detalles SKODA Kamiq (2023),Matriculado: 05/2023,23161,SUZUKI SKODA MOURENTE Pontevedra,Profesional,C,Sí,19990,26783,4241 mm Largo,1793 mm Ancho,1553 mm Alto,Capacidad del maletero (l)40

In [17]:
df.to_csv('../bin/data.csv', index=False)

## Extracción listado marcas

In [44]:
url = 'https://www.autocasion.com/marcas'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

session = requests.Session()

response = session.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")
print(response.status_code)

200


In [66]:
marcas = soup.find('ul', class_= 'marcas').find_all('span')


listado_marcas = {}
for index,detalle in enumerate (marcas, start = 1):
    detalle = detalle.text.strip()
    
    listado_marcas[index] = detalle
    

In [68]:
listado_marcas

{1: 'Abarth',
 2: 'Aiways',
 3: 'Alfa romeo',
 4: 'Alpine',
 5: 'Aston martin',
 6: 'Audi',
 7: 'Bentley',
 8: 'Bmw',
 9: 'Byd',
 10: 'Citroen',
 11: 'Cupra',
 12: 'Dacia',
 13: 'Dfsk',
 14: 'Dongfeng',
 15: 'Dr',
 16: 'Ds',
 17: 'Evo',
 18: 'Ferrari',
 19: 'Fiat',
 20: 'Ford',
 21: 'Honda',
 22: 'Hyundai',
 23: 'Ineos',
 24: 'Invicta',
 25: 'Isuzu',
 26: 'Jaecoo',
 27: 'Jaguar',
 28: 'Jeep',
 29: 'Jiayuan',
 30: 'Kgm / ssangyong',
 31: 'Kia',
 32: 'Lamborghini',
 33: 'Lancia',
 34: 'Land-rover',
 35: 'Leapmotor',
 36: 'Levc',
 37: 'Lexus',
 38: 'Livan',
 39: 'Lotus',
 40: 'Lynk & co',
 41: 'Maserati',
 42: 'Maxus',
 43: 'Mazda',
 44: 'Mercedes-benz',
 45: 'Mg',
 46: 'Mhero',
 47: 'Mini',
 48: 'Mitsubishi',
 49: 'Morgan',
 50: 'Nissan',
 51: 'Omoda',
 52: 'Opel',
 53: 'Peugeot',
 54: 'Polestar',
 55: 'Porsche',
 56: 'Renault',
 57: 'Seat',
 58: 'Seres',
 59: 'Skoda',
 60: 'Smart',
 61: 'Subaru',
 62: 'Suzuki',
 63: 'Swm',
 64: 'Tesla',
 65: 'Toyota',
 66: 'Volkswagen',
 67: 'Volvo',
 6

In [70]:
df_marcas = pd.DataFrame(list(listado_marcas.items()), columns=['id_marca', 'nombre_marca'])

In [72]:
df_marcas.to_csv('../bin/listado_marcas.csv', index=False)